In [1]:
import numpy as np
import pandas as pd
import os, json, operator
from bs4 import BeautifulSoup as BS
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from numpy import asarray as arr

In [2]:
base = os.path.dirname(os.getcwd())

In [3]:
news_loc = os.path.join(base, 'news')
news_dump = os.path.join(base, 'news extracting')

In [4]:
companies = os.listdir(news_loc)

In [5]:
def multi_replace(s, rep_dict):
    for key, value in rep_dict.items():
        s = s.replace(key, value)
        
    return s

In [27]:
z = ['0' + str(x) for x in range(1,10)] + ['10', '11', '12']
z = ['.' + x + '.' for x in z]

months_infl = [' stycznia ', ' lutego ', ' marca ', ' kwietnia ', ' maja ', ' czerwca ', ' lipca ', 
                                    ' sierpnia ', ' września ', ' października ', ' listopada ', ' grudnia ']

month_dict = dict(zip(z, [' styczeń ', ' luty ', ' marzec ', ' kwiecień ', ' maj ', ' czerwiec ', ' lipiec ', 
                         ' sierpień ', ' wrzesień ', ' październik ', ' listopad ', ' grudzień ']))

month_dict_inflection = dict(zip([' styczeń ', ' luty ', ' marzec ', ' kwiecień ', ' maj ', ' czerwiec ', ' lipiec ', 
                         ' sierpień ', ' wrzesień ', ' październik ', ' listopad ', ' grudzień '], months_infl))

month_infl_to_numb = dict(zip(months_infl, [x.replace('.', '') for x in z]))

inflected_months = set([x.strip(' ') for x in months_infl])

infl_months_order = dict(zip([x.strip(' ') for x in months_infl], [x+1 for x in range(12)]))

In [43]:
def retrieve_date(item):
    arrayed = [x for x in item.split(" ") if x != '']
    setted = set(arrayed)
    M = setted & inflected_months
    if not M:
        return None
    else:
        ind = 0
        if len(M) == 1:
            ind = arrayed.index(*M)
        else:
            max_month = ''
            for month in M:
                if infl_months_order[month] > ind:
                    max_month = month
                    ind = infl_months_order[month]
                    
            ind = arrayed.index(max_month)
        
        
        return '-'.join( [arrayed[ind-1], str(infl_months_order[arrayed[ind]]), arrayed[ind+1]] ).rstrip("r.<br/>").lstrip('o')

In [71]:
def retrieve_dates(item): #former_date - format DD-MM-YYYY
    arrayed = [x for x in item.split(" ") if x != '']
    setted = set(arrayed)
    M = setted & inflected_months
    dates = []
    
    for month in M:
        ind = np.where(arr(arrayed) == month)[0] #jesli przekladasz z 13 stycznia na 14 stycznia, to nic nie wykryjesz
        for i in ind:
            dates.append( '-'.join( [ arrayed[i-1], str(infl_months_order[month]), arrayed[i+1] ]))
        
    return dates

In [73]:
t = []
for comp in companies[20:40]:
    
    l = os.path.join(news_loc, comp)
    a = []
    print(comp)
    for file in os.listdir(l):
        if file == 'changes.txt':
            continue
        o = open(os.path.join(l,file), 'r').read()
        try:
            soup = BS(o, 'html.parser').title.string.lower()
            if 'termin' in soup:
                a.append( (file.split(' ')[0], soup, file) )
                
        except:
            print(file)
            continue
        
        
    a = sorted(a, key=operator.itemgetter(0))
#     for annoucement in a:
#         if ('zmiana' in annoucement[1] or 'korekta' in annoucement[1] ) and 'raport' in annoucement[1]:
#             print(annoucement, '\n')
        
    years = sorted(list(set([int(x[0].split('-')[0]) for x in a])))
    K, M = 0, 0
    for year in years:
        f = open(os.path.join(l, a[K][2]), 'r').read()
        print(year, '__________________________', '\n', '\n')
        huge_mess = [x for x in f.split('\n') if ('raport' in x.lower() or 'kwarta' in x.lower() or 'roczn' in x.lower()) and 'Ministr' not in x]
        less_mess = []
        for item in huge_mess:
            item = multi_replace(item, month_dict)
            item = multi_replace(item, month_dict_inflection)
            if '<TITLE>' in item:
                continue
            
            if 'III kw' in item and not 'III.' in item:
                less_mess.append(('III', retrieve_date(item), item))
                
            elif 'II kw' in item and not 'III' in item:
                less_mess.append(('II', retrieve_date(item), item))
            elif 'półrocz' in item:
                less_mess.append(('półroczny', retrieve_date(item), item))
            elif 'roczn' in item:
                less_mess.append(('roczny', retrieve_date(item), item))
            elif 'IV kw' in item or 'roczn' in item:
                less_mess.append(('IV',retrieve_date(item),  item))
            elif 'I kw' in item:
                less_mess.append(('I', retrieve_date(item), item))
        
        d = {'I':[], 'II':[], 'półroczny':[], 'III':[], 'IV':[], 'roczny':[]}
        
        for thing in less_mess:
            if thing[1]:
                d[thing[0]].append((thing[1], thing[2]))
                
        #skonsolidowane sa pozniej, bierzemy skonsolidowane, bo takie mamy raporty
        #mozna sprobowac doparsowac jednostkowe, wiec bedziemy rozwazac zarowno jedne, jak i drugie
        #ale to będzie bardziej niż uciążliwe
            
            
        for key in d.keys():
            proper_nr = -1
            for i, tupl in enumerate(d[key]):
                if 'skonsolidowan' or 'Skonsolidowan' in tupl[1]:
                    proper_nr = i
                    break
            if len(d[key]):
                d[key] = d[key][proper_nr][0]
#             print(key, d[key])
            
        #a teraz zmiany 
        
        print('filename', a[K][2], d)
        while datetime.strptime(a[K][0], '%Y-%m-%d') < datetime(year=year, month=1, day=1):
# tutaj pojdzie sprawdzanie, czy byly jakies zmiany po drodze
            K += 1
            if ('zmiana' in a[K][1] or 'korekta' in a[K][1] ) and 'raport' in a[K][1]:
                f = open(os.path.join(l, a[K][2]), 'r').read()
                dates = retrieve_dates(f)
                print(a[K][2], len(dates), dates, d.values())
                
                
        
        
    
    print('checking for missed years')
    diffs = np.diff([int(x[0].split('-')[0]) for x in a])
    if 2 in diffs:
        mi, ma = min(years), max(years)
        missed = []
        for y in range(len(years)):
            if mi + y not in years:
                missed.append(mi+y)
        print('missed:',  missed)

DECORA
2005 __________________________ 
 

filename 2005-08-02 16:47.txt {'półroczny': '22-9-2005', 'II': '04-8-2005', 'I': [], 'III': '04-11-2005', 'roczny': [], 'IV': []}
2006 __________________________ 
 

filename 2005-08-02 16:47.txt {'półroczny': '22-9-2005', 'II': '04-8-2005', 'I': [], 'III': '04-11-2005', 'roczny': [], 'IV': []}
2007 __________________________ 
 

filename 2006-01-31 08:43.txt {'półroczny': [], 'II': [], 'I': [], 'III': [], 'roczny': [], 'IV': []}
2006-03-21 08:57.txt 3 ['31-1-2006', '2-10-2006', '11-10-2006'] dict_values([[], [], [], [], [], []])
2006-11-07 09:06.txt 3 ['31-1-2006', '9-11-2006', '14-11-2006'] dict_values([[], [], [], [], [], []])
2008 __________________________ 
 

filename 2007-01-30 09:18.txt {'półroczny': '1.-10-2007', 'II': [], 'I': [], 'III': [], 'roczny': '11.-4-2007', 'IV': []}
2007-03-28 17:03.txt 3 ['19-10-2005', '11-4-2007', '19-4-2007'] dict_values(['1.-10-2007', [], [], [], '11.-4-2007', []])
2007-11-09 17:00.txt 3 ['30.-1-2007', '

In [38]:
a = [('20-12-2015', 'ajatollah'), ('24-11-2016', 'sprawdzam')]

sorted(a, key=operator.itemgetter(0))

[('20-12-2015', 'ajatollah'), ('24-11-2016', 'sprawdzam')]